# 1. 데이터 병합

## 1-1. '성북구'행정 코드로 성북구의 상권 코드 찾기

행정동코드와 연결된 코드는 제공하지 않으므로, 상권이 위치한 행정동으로 파악하셔서 이용해 주시기 바랍니다.
행정동코드는 "서울빅데이터 > 서울생활인구 > 자치구 단위 > 행정구역 코드정보"를 다운받아 이용해 주시기 바랍니다.
감사합니다.

- 서울 열린 데이터 광장에서 제공하는 좌표값 기준은 EPSG:5181이다 
<br>
EPSG:5181 → 중부원점(GRS80)
서비스: 다음카카오 지도 API5
CRS 스펙: +proj=tmerc +lat_0=38 +lon_0=127 +k=1 +x_0=200000 +y_0=500000 +ellps=GRS80 +units=m +no_defs

- 지도 표시하기

In [331]:
import numpy as np
import os
import pandas as pd

dir_path='./데이터'
data='서울시 우리마을가게 상권분석서비스(상권영역).csv'
# print(os.listdir(dir_path))

#데이터 불러오기

df=pd.read_csv(dir_path+'/'+data,header=0,engine='python',encoding='cp949')
df=df[['시군구_코드','행정동_코드','상권_코드','상권_구분_코드','상권_코드_명','엑스좌표_값','와이좌표_값']]
# df.head()


#시군구 코드 11290(성북구)만 찾기
df=df.loc[df['시군구_코드'] == 11290]
df=df.reset_index(drop=True) #인덱스 재정렬
df


시군구_코드    행정동_코드    상권_코드 상권_구분_코드       상권_코드_명  엑스좌표_값  와이좌표_값
0    11290  11290770  2110332        A       돌곶이역 3번  205155  457004
1    11290  11290810  2110331        A       돌곶이역 8번  205055  456506
2    11290  11290600  2110305        A        보문역 5번  201963  453846
3    11290  11290525  2110288        A          동구여중  200468  454602
4    11290  11290620  2110298        A       솔숲어린이공원  201401  456115
..     ...       ...      ...      ...           ...     ...     ...
71   11290  11290555  2130120        R  돈암시장(돈암제일시장)  201400  454692
72   11290  11290810  2130127        R          석관시장  205333  456599
73   11290  11290660  2130121        R          길음시장  201944  455945
74   11290  11290715  2130122        R        밤나무골시장  203136  456581
75   11290  11290630  2130119        R       정릉아리랑시장  201050  455910

[76 rows x 7 columns]

In [332]:
#이 데이터프레임을 바탕으로 상권 코드를 찾아보자

col=df['상권_코드'] #11290인 dataframe에서
col_list=col.values.tolist() #리스트에 담기 
print('성북구 상권 코드:',col_list)
len(col_list) #상권 76개 분석

성북구 상권 코드: [2110332, 2110331, 2110305, 2110288, 2110298, 2110320, 2110287, 2110312, 2110330, 2110283, 2110309, 2110302, 2110334, 2110333, 2110313, 2110286, 2110292, 2110289, 2110284, 2110285, 2110314, 2110282, 2110326, 2110324, 2110311, 2110306, 2110318, 2110319, 2110321, 2110299, 2110308, 2110335, 2110336, 2110316, 2110221, 2110310, 2110301, 2110329, 2110328, 2110322, 2110327, 2110281, 2110280, 2110317, 2110291, 2110293, 2110296, 2110295, 2110290, 2110294, 2110307, 2110300, 2110303, 2110304, 2110297, 2110325, 2110315, 2110323, 2120073, 2120075, 2120071, 2120074, 2120070, 2120072, 2130126, 2130125, 2130124, 2130117, 2130118, 2130123, 2130128, 2130120, 2130127, 2130121, 2130122, 2130119]


76

- 성북구 상권 코드 리스트 : col_list

# 새로운 데이터프레임
table

In [403]:
table=df[['상권_코드','상권_코드_명']]
table

상권_코드       상권_코드_명
0   2110332       돌곶이역 3번
1   2110331       돌곶이역 8번
2   2110305        보문역 5번
3   2110288          동구여중
4   2110298       솔숲어린이공원
..      ...           ...
71  2130120  돈암시장(돈암제일시장)
72  2130127          석관시장
73  2130121          길음시장
74  2130122        밤나무골시장
75  2130119       정릉아리랑시장

[76 rows x 2 columns]

## 1-2.상권코드를 기준으로 데이터프레임들을 병합하자

In [350]:
#함수 []
def store_code_df(data):
    df=pd.read_csv(dir_path+'/'+data,header=0,engine='python',encoding='cp949')
    #컬럼명이 '상권 코드이면' ->상권_코드로 바꾸고 ㄱㄱ
    if '상권 코드' in df:
        df=df.rename(columns={'상권 코드':'상권_코드'})
    if '기준 년코드'in df:
        df=df.rename(columns={'기준 년코드':'기준_년_코드'})
    new_df=df[df['상권_코드'].isin(col_list)] #상권코드 리스트 안에 있으면 새로운 데이터프레임 생성
    new_df=new_df.loc[new_df['기준_년_코드'] ==2022] #2022
    new_df=new_df.loc[new_df['기준_분기_코드']==3] #3분기 
    
    
    new_df=new_df.reset_index(drop=True) #인덱스 재정렬
    return new_df



#함수[2022,3분기 컬럼 제거]
def drop_col(df):
    new_df=df.drop(columns=['기준_년_코드', '기준_분기_코드','상권_구분_코드','상권_구분_코드_명'])
    return new_df


### (1)집객시설

In [351]:
data='서울시 우리마을가게 상권분석서비스(상권배후지-집객시설).csv'

# df1
df1=store_code_df(data) #code1
df1=drop_col(df1)

df1

상권_코드       상권_코드_명  집객시설_수  관공서_수  은행_수  종합병원_수  일반_병원_수  약국_수  유치원_수  \
0   2110321          동덕여대      66    2.0   NaN     NaN      NaN   4.0    2.0   
1   2110335        석계역 7번     112    4.0   2.0     NaN      NaN  11.0    NaN   
2   2110329       우리은행체육관      72    1.0   NaN     NaN      NaN   9.0    2.0   
3   2110322      장위1동주민센터      72    3.0   1.0     NaN      NaN   4.0    6.0   
4   2110293         삼선중학교     173    5.0   9.0     NaN      NaN  20.0    3.0   
5   2110299        정릉역 1번     153    4.0   5.0     NaN      NaN  16.0    4.0   
6   2110295         정릉우체국     144    7.0   5.0     NaN      NaN  17.0    3.0   
7   2110307        길음역 7번     100    3.0   4.0     NaN      NaN   9.0    NaN   
8   2110313        안암역 3번     164    3.0   6.0     1.0      2.0  12.0    NaN   
9   2110306        미아초등학교      66    2.0   1.0     NaN      NaN   8.0    1.0   
10  2110304        보문역 4번     139    5.0   5.0     1.0      NaN   7.0    NaN   
11  2110300       삼선동주민센터     192    5.0   8.0     1.0      NaN  15.0    1.0   
12  2110316        숭곡초등학교     214    5.0   4.0     NaN      3.0  25.0    1.0   
13  2110327       상월곡역 2번      57    4.0   1.0     NaN      NaN   4.0    NaN   
14  2110221            의릉      38    1.0   NaN     NaN      NaN   5.0    NaN   
15  2110330       돌곶이역 1번      70    4.0   2.0     NaN      NaN   7.0    NaN   
16  2110315      종암아이파크2차     105    6.0   2.0     NaN      1.0  11.0    3.0   
17  2110308        길음역 8번     133    7.0   2.0     NaN      1.0  15.0    1.0   
18  2110296    성신여대입구역 4번     156    5.0   7.0     NaN      1.0  14.0    3.0   
19  2110287      정릉3동주민센터      80    3.0   1.0     NaN      NaN   1.0    NaN   
20  2110285          홍대부중      68    4.0   2.0     NaN      NaN   NaN    1.0   
21  2110311    종암아이파크2차후문      34    2.0   NaN     NaN      NaN   NaN    1.0   
22  2110323       상월곡역 1번     102    5.0   2.0     NaN      NaN   8.0    2.0   
23  2110324        월곡역 4번      84    5.0   3.0     NaN      NaN   9.0    2.0   
24  2110288          동구여중     125    4.0   3.0     NaN      NaN   9.0    3.0   
25  2110280          북정마을      78    2.0   2.0     NaN      NaN   1.0    1.0   
26  2110302        보문역 8번      66    3.0   3.0     NaN      NaN   5.0    NaN   
27  2110294         서경대학교     130    6.0   4.0     NaN      NaN  13.0    2.0   
28  2110281           길상사      62    2.0   1.0     NaN      NaN   NaN    1.0   
29  2110314         종암경찰서     132    8.0   2.0     NaN      NaN  15.0    3.0   
30  2110283       성북초등학교앞      77    3.0   2.0     NaN      NaN   NaN    2.0   
31  2110298       솔숲어린이공원      70    4.0   2.0     NaN      NaN  10.0    2.0   
32  2110333       돌곶이역 5번     108    4.0   3.0     NaN      NaN  15.0    1.0   
33  2110289    북한산보국문역 2번     157   11.0   4.0     NaN      NaN  12.0    NaN   
34  2110284   정릉북한산국립공원입구      30    NaN   NaN     NaN      NaN   2.0    1.0   
35  2110310       길음역 10번      84    4.0   NaN     NaN      1.0   9.0    NaN   
36  2110317         종암중학교      98    3.0   2.0     NaN      NaN   3.0    1.0   
37  2110309        대광초등학교      90    4.0   4.0     NaN      1.0   5.0    NaN   
38  2110297        경동고등학교     172    8.0   5.0     NaN      NaN  20.0    NaN   
39  2110318        KT월곡지사     121    3.0   2.0     NaN      1.0  19.0    NaN   
40  2110334        석관중앙교회      73    3.0   2.0     NaN      NaN  12.0    1.0   
41  2110331       돌곶이역 8번     101    3.0   3.0     NaN      NaN  14.0    1.0   
42  2110336        석관고등학교      43    4.0   NaN     NaN      NaN   4.0    NaN   
43  2110291     한성대입구역 3번     145    4.0   5.0     NaN      NaN  12.0    NaN   
44  2110286     한성대입구역 5번     133    7.0   3.0     NaN      NaN  11.0    4.0   
45  2110301    성신여대입구역 7번     147    1.0   4.0     NaN      1.0  15.0    4.0   
46  2110305        보문역 5번     125    5.0   5.0     1.0      NaN   7.0    NaN   
47  2110326         장위동성당      74    2.0   1.0     NaN      NaN   7.0    1.0   
48  2110290    북한산보국문역 1번      79    8.0   1.0     NaN      NaN   4.0    1.0   
49  21

### (2)상주 인구

In [352]:
data='서울시 우리마을가게 상권분석서비스(상권-상주인구).csv'

#데이터 불러오기

df2=store_code_df(data)
df2=drop_col(df2)
df2

상권_코드  총 상주인구 수  남성 상주인구 수  여성 상주인구 수  연령대 10 상주인구 수  연령대 20 상주인구 수  \
0   2130128      1776        918        858            162            221   
1   2130127       150         82         68             10             24   
2   2130126        55         27         28              1             10   
3   2130125       839        407        432             87             95   
4   2130124        16          8          8              5              0   
..      ...       ...        ...        ...            ...            ...   
71  2110283       501        209        292             60             74   
72  2110282      1907        989        918            211            496   
73  2110281      1055        492        563            146            139   
74  2110280       840        400        440             64            102   
75  2110221      1028        514        514            118            220   

    연령대 30 상주인구 수  연령대 40 상주인구 수  연령대 50 상주인구 수  연령대 60 이상 상주인구 수  ...  \
0             198            256            325               614  ...   
1              12             16             35                53  ...   
2               4              6             12                22  ...   
3              93            116            146               302  ...   
4               1              3              1                 6  ...   
..            ...            ...            ...               ...  ...   
71             55             80             93               139  ...   
72            232            211            235               522  ...   
73            114            166            190               300  ...   
74            111            112            147               304  ...   
75            146            157            168               219  ...   

    여성연령대 10 상주인구 수  여성연령대 20 상주인구 수  여성연령대 30 상주인구 수  여성연령대 40 상주인구 수  \
0                86               99               83              116   
1                 2               10                5                6   
2                 0                6                2                1   
3                45               42               48               47   
4                 2                0                1                2   
..              ...              ...              ...              ...   
71               32               49               29               46   
72               97              226               94               96   
73               67               79               66               95   
74               32               55               56               55   
75               54              115               74               69   

    여성연령대 50 상주인구 수  여성연령대 60 이상 상주인구 수  총 가구 수  아파트 가구 수  비 아파트 가구 수  \
0               145                 329     924         0         924   
1                14                  31      87         0          87   
2                 5                  14      34         0          34   
3                77                 173     397         0         397   
4                 1                   2       8         0           8   
..              ...                 ...     ...       ...         ...   
71               51                  85     244         0         244   
72              108                 297    1131         0        1131   
73               97                 159     443         0         443   
74               71                 171     459         0         459   
75               85                 117     565         0         565   

          상권 코드 명  
0   돌곶이시장(석관황금시장)  
1            석관시장  
2           새석관시장  
3          장위전통시장  
4            번동시장  
..            ...  
71        성북초등학교앞  
72         국민대학교앞  
73            길상사  
74           북정마을  
75             의릉  

[76 rows x 26 columns]

### (3) 생활인구

In [362]:
data='서울시 우리마을가게 상권분석서비스(상권-생활인구).csv'

df3=store_code_df(data) #code1
df3=drop_col(df3)
df3


상권_코드        상권_코드_명  총_생활인구_수  남성_생활인구_수  여성_생활인구_수  연령대_10_생활인구_수  \
0   2130128  돌곶이시장(석관황금시장)    300594     137634     162959          41715   
1   2130127           석관시장     38157      17269      20888           6357   
2   2130126          새석관시장     32530      14986      17545           4492   
3   2130125         장위전통시장    330852     140950     189902          49492   
4   2130124           번동시장     12333       5704       6629           2544   
..      ...            ...       ...        ...        ...            ...   
71  2110283        성북초등학교앞    157871      72592      85279          33221   
72  2110282         국민대학교앞     98979      53217      45762          13207   
73  2110281            길상사     66547      30963      35583           6478   
74  2110280           북정마을     80916      34670      46246           8919   
75  2110221             의릉    254095     119713     134382          39939   

    연령대_20_생활인구_수  연령대_30_생활인구_수  연령대_40_생활인구_수  연령대_50_생활인구_수  ...  \
0           41459          42156          43366          50494  ...   
1            4477           4511           5269           6068  ...   
2            4866           4754           4852           5168  ...   
3           35436          38813          43464          54144  ...   
4            1171           1795           2074           1758  ...   
..            ...            ...            ...            ...  ...   
71          18316          24113          25482          24447  ...   
72          30326          12996           9227          10814  ...   
73           6715           7979          11571          13920  ...   
74           7681          10967          13691          14836  ...   
75          38816          28110          43323          35704  ...   

    여성연령대_60_이상_토요일시간대_3_생활인구_수  여성연령대_60_이상_토요일시간대_4_생활인구_수  \
0                           779                          748   
1                           118                          112   
2                            82                           82   
3                          1254                         1314   
4                            33                           33   
..                          ...                          ...   
71                          367                          360   
72                          274                          228   
73                          233                          203   
74                          256                          244   
75                          871                          884   

    여성연령대_60_이상_토요일시간대_5_생활인구_수  여성연령대_60_이상_토요일시간대_6_생활인구_수  \
0                           961                         1046   
1                           150                          132   
2                           120                           92   
3                          1802                         1267   
4                            48                           32   
..                          ...                          ...   
71                          506                          440   
72                          344                          230   
73                          271                          212   
74                          315                          281   
75                         1245                          792   

    여성연령대_60_이상_일요일시간대_1_생활인구_수  여성연령대_60_이상_일요일시간대_2_생활인구_수  \
0                          2353                         1752   
1                           299                          219   
2                           161                          146   
3                          2513                         2355   
4                            54                           46   
..                          ...                          ...   
71                          697                          642   
72                          552                          499   
73                          432                          379   
74                          619                   

### (4)소득-소비

In [364]:
data='서울시 우리마을가게 상권분석서비스(상권-소득소비).csv'


df4=store_code_df(data) #code1
df4 = drop_col(df4) 
df4

상권_코드        상권_코드_명  월_평균_소득_금액  소득_구간_코드       지출_총금액   식료품_지출_총금액  \
0   2110221             의릉   2325126.0       5.0  792796632.0  201748887.0   
1   2110280           북정마을   3358357.0       7.0  472892235.0  131732168.0   
2   2110281            길상사   5608226.0       9.0  850269857.0  208901738.0   
3   2110282         국민대학교앞   2787829.0       6.0  854374378.0  230982081.0   
4   2110283        성북초등학교앞   3079970.0       7.0  186734062.0   51563645.0   
..      ...            ...         ...       ...          ...          ...   
71  2130124           번동시장   4771734.0       8.0   25527557.0    7494413.0   
72  2130125         장위전통시장   2356847.0       5.0  583325575.0  177697406.0   
73  2130126          새석관시장   2396735.0       5.0   74660626.0   16923166.0   
74  2130127           석관시장   2565610.0       6.0   49776460.0   14921092.0   
75  2130128  돌곶이시장(석관황금시장)   2156814.0       5.0  583569468.0  197330305.0   

    의류_신발_지출_총금액  생활용품_지출_총금액   의료비_지출_총금액    교통_지출_총금액   여가_지출_총금액  \
0     97412195.0   58883451.0   97262889.0  131867261.0  30283049.0   
1     55714681.0   34353669.0   65615344.0   69892878.0  18428580.0   
2    105080670.0   64550470.0  110872100.0  147359026.0  36132062.0   
3    105811389.0   70028348.0  109409802.0  135132214.0  34672854.0   
4     21831891.0   13335421.0   24360729.0   28562518.0   5921460.0   
..           ...          ...          ...          ...         ...   
71     3054365.0    1951268.0    3769106.0    4019204.0    829858.0   
72    67292455.0   40223831.0   89285319.0   84268008.0  19405022.0   
73    10183718.0    5758098.0    8641158.0   13374268.0   4435040.0   
74     5941203.0    3575692.0    7336923.0    7418719.0   1698588.0   
75    64013254.0   40512673.0   96115463.0   71045788.0  16682328.0   

     문화_지출_총금액    교육_지출_총금액   유흥_지출_총금액  
0   24421289.0  116927808.0  33989803.0  
1   14460307.0   64630755.0  18063853.0  
2   23913848.0  117775639.0  35684304.0  
3   28428716.0  103091662.0  36817312.0  
4    5917806.0   27454828.0   7785764.0  
..         ...          ...         ...  
71    793269.0    2442797.0   1173277.0  
72  17924949.0   64109836.0  23118749.0  
73   2109065.0   10086694.0   3149419.0  
74   1620125.0    5143600.0   2120518.0  
75  19813678.0   56138989.0  21916990.0  

[76 rows x 14 columns]

### (5) 직장인구

In [368]:
data='서울시 우리마을가게 상권분석서비스(상권-직장인구).csv'

df5=store_code_df(data) #code1
df5=drop_col(df5)
df5

상권_코드        상권_코드_명  총_직장_인구_수  남성_직장_인구_수  여성_직장_인구_수  연령대_10_직장_인구_수  \
0   2130128  돌곶이시장(석관황금시장)        191         157          34               0   
1   2130126          새석관시장          3           1           2               0   
2   2130125         장위전통시장         40          20          20               0   
3   2130123  꿈의숲 장곡 골목형상점가          6           5           1               0   
4   2130122         밤나무골시장         17           5          12               0   
..      ...            ...        ...         ...         ...             ...   
69  2110283        성북초등학교앞        203          78         125               0   
70  2110282         국민대학교앞       1403        1162         241              15   
71  2110281            길상사      13811       12087        1724              12   
72  2110280           북정마을         78          40          38               0   
73  2110221             의릉         95          69          26               0   

    연령대_20_직장_인구_수  연령대_30_직장_인구_수  연령대_40_직장_인구_수  연령대_50_직장_인구_수  ...  \
0               30              36              41              72  ...   
1                0               0               1               2  ...   
2                5               6               7               8  ...   
3                1               0               1               4  ...   
4                2               4               6               4  ...   
..             ...             ...             ...             ...  ...   
69              16              33              56              80  ...   
70             213             430             341             239  ...   
71            1418            3093            4057            4963  ...   
72               6              12              25              24  ...   
73              10              12              36              29  ...   

    남성연령대_30_직장_인구_수  남성연령대_40_직장_인구_수  남성연령대_50_직장_인구_수  남성연령대_60_이상_직장_인구_수  \
0                 26                29                62                   12   
1                  0                 0                 1                    0   
2                  2                 5                 5                    8   
3                  0                 1                 3                    0   
4                  1                 2                 1                    0   
..               ...               ...               ...                  ...   
69                10                23                27                   10   
70               387               296               212                  102   
71              2426              3678              4822                  200   
72                 4                11                15                    8   
73                10                24                22                    7   

    여성연령대_10_직장_인구_수  여성연령대_20_직장_인구_수  여성연령대_30_직장_인구_수  여성연령대_40_직장_인구_수  \
0                  0                 2                10                12   
1                  0                 0                 0                 1   
2                  0                 5                 4                 2   
3                  0                 0                 0                 0   
4                  0                 1                 3                 4   
..               ...               ...               ...               ...   
69                 0                 8                23                33   
70                 1                62                43                45   
71                11               458               667               379   
72                 0                 4                 8                14   
73                 0                 4                 2                12   

    여성연령대_50_직장_인구_수  여성연령대_60_이상_직장_인구_수  
0                 10                    0  
1                  1                    0  
2                  3                    6  
3                  1                    0  
4 

### (6) 점포 추정 매출

In [372]:
data='서울시 우리마을가게 상권분석서비스(상권배후지-추정매출).csv'

df6=store_code_df(data) #code1
df6=drop_col(df6)
df6

상권_코드 상권_코드_명 서비스_업종_코드 서비스_업종_코드_명   분기당_매출_금액  분기당_매출_건수  주중_매출_비율  \
0     2110336  석관고등학교  CS300033         철물점    84516068       4276        97   
1     2110336  석관고등학교  CS300032        가전제품    76405007        265        65   
2     2110336  석관고등학교  CS300029        애완동물     4332726        262        77   
3     2110336  석관고등학교  CS300027        섬유제품    57944000       1353        92   
4     2110336  석관고등학교  CS300026          완구     1112831         36       100   
...       ...     ...       ...         ...         ...        ...       ...   
2444  2110221      의릉  CS100007       치킨전문점    87622434       3235        66   
2445  2110221      의릉  CS100006      패스트푸드점      144949         21         1   
2446  2110221      의릉  CS100005         제과점   140830482      13123        68   
2447  2110221      의릉  CS100002       중식음식점     2102711        200        85   
2448  2110221      의릉  CS100001       한식음식점  1897666212      42390        67   

      주말_매출_비율  월요일_매출_비율  화요일_매출_비율  ...  시간대_건수~24_매출_건수  남성_매출_건수  \
0            3          7          3  ...                0      3201   
1           35         38          0  ...                0       194   
2           24         23         18  ...                0        58   
3            8         13         12  ...                0       234   
4            0         18         62  ...                0        22   
...        ...        ...        ...  ...              ...       ...   
2444        34         12         12  ...              772      1838   
2445        99          0          1  ...                0        14   
2446        32         11         12  ...             1351      5381   
2447        15         33          0  ...                0       157   
2448        33         12         14  ...             5591     24951   

      여성_매출_건수  연령대_10_매출_건수  연령대_20_매출_건수  연령대_30_매출_건수  연령대_40_매출_건수  \
0          763             0           478           526           948   
1           71             0            52            36            17   
2          197             0           123             6            25   
3         1072             0            47            47             0   
4           14             0             0             8             0   
...        ...           ...           ...           ...           ...   
2444      1222            84           646           731           608   
2445         7             0             0             0            14   
2446      7682           305          3098          2633          2411   
2447        29             0            28            57            43   
2448     15520           216          3227          6436          7842   

      연령대_50_매출_건수  연령대_60_이상_매출_건수  점포수  
0              528             1483    8  
1               89               71    4  
2               38               63    *  
3              326              886    8  
4               14               14    3  
...            ...              ...  ...  
2444           710              280    8  
2445             7                0    5  
2446          3044             1572    5  
2447            58                0    *  
2448         12446            10306   42  

[2449 rows x 76 columns]

In [373]:
#서비스_업종_코드 보기
df6['서비스_업종_코드_명'].unique()

array(['철물점', '가전제품', '애완동물', '섬유제품', '완구', '화장품', '문구', '의약품', '안경',
       '신발', '일반의류', '반찬가게', '청과상', '수산물판매', '육류판매', '핸드폰', '편의점', '슈퍼마켓',
       '노래방', '여관', '세탁소', '피부관리실', '네일숍', '미용실', '자동차미용', '자동차수리',
       '스포츠클럽', '당구장', '한의원', '치과의원', '일반의원', '스포츠 강습', '예술학원', '외국어학원',
       '일반교습학원', '커피-음료', '호프-간이주점', '분식전문점', '치킨전문점', '패스트푸드점', '제과점',
       '중식음식점', '한식음식점', '인테리어', '운동/경기용품', '서적', '의료기기', '시계및귀금속',
       '미곡판매', '가전제품수리', 'PC방', '양식음식점', '일식음식점', '조명용품', '가구', '화초',
       '골프연습장', '컴퓨터및주변장치판매', '전자상거래업', '자전거 및 기타운송장비', '가방', '고시원'],
      dtype=object)

### Q. 성북구에서 가장 점포 수가 많은 서비스 업종은 무엇일까?

In [412]:
# 서비스업종 별로 

df_sort=df6.groupby(['상권_코드','서비스_업종_코드_명']).agg({'점포수':'sum'})

#상위10개 값 가져와서 데이터프레임으로 만들기
df_sort_top10=df_sort.sort_values(by='점포수',ascending=False).head(10)
df_sort_top10

# #table과 비교해서<-상권코드명 붙이기
t=pd.merge(df_sort_top10,table,on='상권_코드')
t
#상권_코드 


상권_코드 점포수     상권_코드_명
0  2110312  98      안암역 2번
1  2110334  97      석관중앙교회
2  2110309  95      대광초등학교
3  2110294  92       서경대학교
4  2110294   9       서경대학교
5  2110301  91  성신여대입구역 7번
6  2110336  90      석관고등학교
7  2110285   9        홍대부중
8  2110317   9       종암중학교
9  2110304   9      보문역 4번

# 테이블에 붙이기

In [315]:
# # TABLE = pd.merge(table,df1, how='outer',on=['상권_코드','상권_코드_명'])
# # TABLE


# from functools import reduce

df_list = [table,df1,df2,df3,df4] #df5 일단 보류!!
reduce(lambda left, right: pd.merge(left,right,on=['상권_코드']),df_list)



C:\Users\yeseolko\AppData\Local\Temp\ipykernel_25032\2746088068.py:8: FutureWarning: Passing 'suffixes' which cause duplicate columns {'상권_코드_명_x', '상권_구분_코드_x'} in the result is deprecated and will raise a MergeError in a future version.
  reduce(lambda left, right: pd.merge(left,right,on=['상권_코드']),df_list)


상권_코드     상권_코드_명_x  기준_년_코드  기준_분기_코드 상권_구분_코드_x 상권_구분_코드_명_x  \
0   2110332       돌곶이역 3번     2022         3          A         골목상권   
1   2110331       돌곶이역 8번     2022         3          A         골목상권   
2   2110305        보문역 5번     2022         3          A         골목상권   
3   2110288          동구여중     2022         3          A         골목상권   
4   2110298       솔숲어린이공원     2022         3          A         골목상권   
5   2110320  KB국민은행 종암동지점     2022         3          A         골목상권   
6   2110287      정릉3동주민센터     2022         3          A         골목상권   
7   2110312        안암역 2번     2022         3          A         골목상권   
8   2110330       돌곶이역 1번     2022         3          A         골목상권   
9   2110283       성북초등학교앞     2022         3          A         골목상권   
10  2110309        대광초등학교     2022         3          A         골목상권   
11  2110302        보문역 8번     2022         3          A         골목상권   
12  2110334        석관중앙교회     2022         3          A         골목상권   
13  2110333       돌곶이역 5번     2022         3          A         골목상권   
14  2110313        안암역 3번     2022         3          A         골목상권   
15  2110286     한성대입구역 5번     2022         3          A         골목상권   
16  2110292        정릉역 2번     2022         3          A         골목상권   
17  2110289    북한산보국문역 2번     2022         3          A         골목상권   
18  2110284   정릉북한산국립공원입구     2022         3          A         골목상권   
19  2110285          홍대부중     2022         3          A         골목상권   
20  2110314         종암경찰서     2022         3          A         골목상권   
21  2110282        국민대학교앞     2022         3          A         골목상권   
22  2110326         장위동성당     2022         3          A         골목상권   
23  2110324        월곡역 4번     2022         3          A         골목상권   
24  2110311    종암아이파크2차후문     2022         3          A         골목상권   
25  2110306        미아초등학교     2022         3          A         골목상권   
26  2110318        KT월곡지사     2022         3          A         골목상권   
27  2110319       종암동주민센터     2022         3          A         골목상권   
28  2110321          동덕여대     2022         3          A         골목상권   
29  2110299        정릉역 1번     2022         3          A         골목상권   
30  2110308        길음역 8번     2022         3          A         골목상권   
31  2110335        석계역 7번     2022         3          A         골목상권   
32  2110336        석관고등학교     2022         3          A         골목상권   
33  2110316        숭곡초등학교     2022         3          A         골목상권   
34  2110221            의릉     2022         3          A         골목상권   
35  2110310       길음역 10번     2022         3          A         골목상권   
36  2110301    성신여대입구역 7번     2022         3          A         골목상권   
37  2110329       우리은행체육관     2022         3          A         골목상권   
38  2110328       상월곡역 3번     2022         3          A         골목상권   
39  2110322      장위1동주민센터     2022         3          A         골목상권   
40  2110327       상월곡역 2번     2022         3          A         골목상권   
41  2110281           길상사     2022         3          A         골목상권   
42  2110280          북정마을     2022         3          A         골목상권   
43  2110317         종암중학교     2022         3          A         골목상권   
44  2110291     한성대입구역 3번     2022         3          A         골목상권   
45  2110293         삼선중학교     2022         3          A         골목상권   
46  2110296    성신여대입구역 4번     2022         3          A         골목상권   
47  2110295         정릉우체국     2022         3          A         골목상권   
48  2110290    북한산보국문역 1번     2022         3          A         골목상권   
49  2110294         서경대학교     2022         3          A         골목상권   
50  2110307        길음역 7번     2022         3          A         골목상권   
51  2110300       삼선동주민센터     2022         3          A         골목상권   
52  2110303    성신여대입구역 1번     2022         3          A         골목상권   
53  2110304        보문역 4번     2022         3          A         골목상권   
54  2110297        경동고등학교     2022         3  

# [군집화]